## Problem 2

### 讀取資料

In [95]:
from gurobipy import *
import pandas as pd
import numpy as np

In [96]:
# 不同instance，只要更改OR109-2_case02_data_s1.xlsx 中的s1就好
file_path="/Users/terrylu/Desktop/OR gurobi/case study2/OR109-2_student/data/OR109-2_case02_data_s1.xlsx"

In [97]:
# Read excel sheets and transform them into lists and matrices
demand_info=pd.read_excel(file_path,'Demand',index_col = 0)
initial_inventory_info=pd.read_excel(file_path,'Initial inventory',index_col = 0)
shipping_cost_info=pd.read_excel(file_path,'Shipping cost',index_col = 0)
intransit=pd.read_excel(file_path,'In-transit',index_col = 0)
size_info=pd.read_excel(file_path,'Size',index_col = 0)
price_and_cost_info=pd.read_excel(file_path,'Price and cost',index_col = 0)
shortage_info=pd.read_excel(file_path,'Shortage',index_col = 0)

In [98]:
vendor_product=pd.read_excel(file_path,'Vendor-Product',index_col = 0)
vendor_cost=pd.read_excel(file_path,'Vendor cost',index_col = 0)
bounds=pd.read_excel(file_path,'Bounds',index_col = 0)
conflict=pd.read_excel(file_path,'Conflict',index_col = 0)

In [99]:
size=size_info['Size'].values.tolist()

In [100]:
shipping_cost_info['Ocean_freight']=0

In [101]:
shipping_cost=shipping_cost_info.values.tolist()

In [102]:
backorder=shortage_info['Backorder'].values.tolist()
lost_sales=shortage_info['Lost sales'].values.tolist()
backorder_percentage=shortage_info['Backorder percentage'].values.tolist()

In [103]:
demand=demand_info.values.tolist()
initial_inventory=initial_inventory_info['Initial inventory'].values.tolist()

In [104]:
vendor_product=pd.get_dummies(vendor_product["Vendor"])
vendor_product

,1,2
Product,,
1,1,0
2,1,0
3,1,0
4,1,0
5,1,0
...,...,...
96,0,1
97,0,1
98,0,1


In [105]:
vendor_product=vendor_product.values.tolist()
vendor_cost=vendor_cost.values.tolist()
bounds=bounds.values.tolist()
conflict=conflict.values.tolist()

In [106]:
purchasing_cost=price_and_cost_info['Purchasing cost'].values.tolist()
holding_cost=price_and_cost_info['Holding cost'].values.tolist()
fix_cost=[100,80,50]

In [107]:
intransit[1]=initial_inventory

In [108]:
for i in range(4,27):
    intransit[i]=0

In [109]:
intransit

,1,2,3,4,5,6,7,8,9,10,...,17,18,19,20,21,22,23,24,25,26
Product,,,,,,,,,,,,,,,,,,,,,
1,800,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,600,48,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,425,0,20,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,350,153,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,400,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,577,0,148,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,915,0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,790,150,99,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [110]:
intransit=intransit.values.tolist()

In [111]:
product_len=range(len(demand))
month_len=range(26)
shipping_method_len=range(3)
vendor_len=range(len(vendor_cost))
conflict_len=range(len(conflict))

In [112]:
C=1500
V=0.5

In [113]:
# Big number M
M=np.sum(demand)
M=int(M)

### 模型

In [114]:
cs2 = Model("cs2")

In [115]:
#定義variable
#定義x_ijt
x = []
for i in product_len:
    x.append([])
    for j in shipping_method_len:
        x[i].append([])
        for t in month_len:
            x[i][j].append(cs2.addVar(lb = 0, vtype = GRB.INTEGER, 
                                      name = "x" +  str(i+1) +'_'+ str(j+1)+'_'+ str(t+1))) 
            
#定義v_it
v = []
for i in product_len:
    v.append([])
    for t in month_len:
        v[i].append(cs2.addVar(lb = 0, vtype = GRB.INTEGER,
                                  name = "v" +  str(i+1)+ '_'+ str(t+1))) 
        
#定義y_jt
y = []
for j in shipping_method_len:
    y.append([])
    for t in month_len:
        y[j].append(cs2.addVar(lb = 0, vtype = GRB.BINARY, 
                                  name = "y" +  str(j+1)+ '_'+ str(t+1))) 
        
#定義z
z = []
for t in month_len:
    z.append(cs2.addVar(lb = 0, vtype = GRB.INTEGER, 
                              name = "z" +'_'+ str(t+1))) 
    
#定義s_it
s = []
for i in product_len:
    s.append([])
    for t in month_len:
        s[i].append(cs2.addVar(lb = 0, vtype = GRB.INTEGER,
                                  name = "s" +  str(i+1)+ '_'+ str(t+1))) 
        
#定義k_it
k_it = []
for i in product_len:
    k_it.append([])
    for t in month_len:
        k_it[i].append(cs2.addVar(lb = 0, vtype = GRB.BINARY, 
                                  name = "k" +  str(i+1)+ '_'+ str(t+1))) 
        
#定義D_it
D = []
for i in product_len:
    D.append([])
    for t in month_len:
        D[i].append(cs2.addVar(lb = 0, vtype = GRB.INTEGER,
                                  name = "D" +  str(i+1)+ '_'+ str(t+1))) 
        
#定義w_kt
w = []
for k in vendor_len:
    w.append([])
    for t in month_len:
        w[k].append(cs2.addVar(lb = 0, vtype = GRB.BINARY, 
                                  name = "w" +  str(k+1)+ '_'+ str(t+1))) 
        
#定義b_it
b = []
for i in product_len:
    b.append([])
    for t in month_len:
        b[i].append(cs2.addVar(lb = 0, vtype = GRB.BINARY, 
                                  name = "b" +  str(i+1)+ '_'+ str(t+1)))

In [116]:
# setting the objective function 
cs2.setObjective(
    
    np.sum( [[v[i][t]*holding_cost[i] for i in product_len] for t in month_len])+  # inventory*holding cost
    np.sum([(np.array(shipping_cost)+np.array(purchasing_cost)[:,None])*np.array(x)[:,:,t] for t in month_len])+ #x's shipping and purchasing cost
    np.sum(np.array(y)*np.array(fix_cost)[:,None])+ # method's fix cost
    np.sum(np.array(z)*C )+ # container  cost
    np.sum(((1-np.array(backorder_percentage))[:,None]*np.array(lost_sales)[:,None])*s)+ #cost of lost sales
    np.sum((np.array(backorder_percentage)[:,None]*np.array(backorder)[:,None])*s)+ #cost of back order
    np.sum(np.array(w)*np.array(vendor_cost)) #vendor fix cost
    
    , GRB.MINIMIZE) 

In [117]:
cs2.addConstrs( ( v[i][t]-s[i][t] == v[i][t-1]+ (np.array(x)[i,:,t]).sum()+ intransit[i][t]-D[i][t]
                 for i in product_len for t in month_len if t>0), "inventory demand equation" )
cs2.addConstrs( ( v[i][t]-s[i][t] == (np.array(x)[i,:,t]).sum()+ intransit[i][0]-D[i][t]
                 for i in product_len for t in month_len if t==0), "inventory demand equation first round" )

cs2.addConstrs( ( M*k_it[i][t] >= s[i][t] for i in product_len for t in month_len), "k is bigger than s" )
cs2.addConstrs( ( M*(1-k_it[i][t]) >= v[i][t] for i in product_len for t in month_len), "1-k is bigger than v" )
cs2.addConstrs( ( M*y[j][t] >= (np.array(x)[:,j,t]).sum()
                 for j in shipping_method_len for t in month_len), "y is bigger than x" )
cs2.addConstrs( ( z[t] >= ((( np.array(x)[:,2,:] * np.array(size)[:,None] ).sum(axis=0) )/V) [t] for t in month_len)
               , "z calculate container number" )

cs2.addConstrs( ( D[i][t] >= demand[i][t]+ backorder_percentage[i]*s[i][t-1] 
                 for i in product_len for t in month_len if t>0), "D >= D" )
cs2.addConstrs( ( D[i][t] >= demand[i][t]+ backorder_percentage[i]*0
                 for i in product_len for t in month_len if t==0), "D >= D first round" )

#case study2 new part
cs2.addConstrs( ( M*w[k][t] >= np.sum((np.array(vendor_product))[:,k]*(np.array(x)[:,:,t].sum(axis=1)))
                 for k in vendor_len for t in month_len), "w is bigger than W*x" )


cs2.addConstrs( ( M*b[i][t] >= (np.array(x)[i,:,t]).sum()
                for i in product_len for t in month_len), "b is bigger than x" )
cs2.addConstrs( ( b[i][t] <= M*((np.array(x)[i,:,t]).sum())
                 for i in product_len for t in month_len), "b is smaller than x" )


#cs2.addConstrs( ( (np.array(x)[i,:,t]).sum() >= (bounds[i][0])*(b[i][t])
                 #for i in product_len for t in month_len), "x is bigger than B" )

cs2.addConstrs((quicksum(x[i][j][t] for j in shipping_method_len) >= bounds[i][0]*b[i][t]
               for i in product_len for t in month_len), "new4_cons")
                 
#conclict constraints
cs2.addConstrs( (  b[conflict[n][0]-1][t]+ b[conflict[n][1]-1][t] <= 1
                 for n in conflict_len for t in month_len ), "conflict")

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (0, 15): <gurobi.Constr *Awaiting Model Update*>,
 (0, 16): <gurobi.Constr *Awaiting Model Update*>,
 (0, 17): <gurobi.Constr *Awaiting Model Update*>,
 (0, 18): <gurobi.Constr *Awaiting Model Update*>,
 (0, 19): <gurobi.Constr *Awaiting Model 

In [118]:
# set the time limit
cs2.setParam('TimeLimit', 10*60)

Changed value of parameter TimeLimit to 600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


In [119]:
cs2.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 18590 rows, 20956 columns and 82017 nonzeros
Model fingerprint: 0x94efbb9d
Variable types: 0 continuous, 20956 integer (5330 binary)
Coefficient statistics:
  Matrix range     [1e-02, 3e+05]
  Objective range  [1e+01, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+05]
Presolve removed 5403 rows and 2803 columns
Presolve time: 0.17s
Presolved: 13187 rows, 18153 columns, 67389 nonzeros
Variable types: 0 continuous, 18153 integer (5232 binary)

Root relaxation: objective 1.287311e+09, 8433 iterations, 0.10 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.2873e+09    0  505          - 1.2873e+09      -     -    0s
H    0     0                    1.361632e+09 1.2873e+09  5.46%     -    0s
H   

In [120]:
print("z* =", cs2.objVal)    # print objective value

z* = 1288944865.5


In [121]:
x

[[[<gurobi.Var x1_1_1 (value -0.0)>,
   <gurobi.Var x1_1_2 (value -0.0)>,
   <gurobi.Var x1_1_3 (value -0.0)>,
   <gurobi.Var x1_1_4 (value -0.0)>,
   <gurobi.Var x1_1_5 (value -0.0)>,
   <gurobi.Var x1_1_6 (value -0.0)>,
   <gurobi.Var x1_1_7 (value -0.0)>,
   <gurobi.Var x1_1_8 (value -0.0)>,
   <gurobi.Var x1_1_9 (value -0.0)>,
   <gurobi.Var x1_1_10 (value -0.0)>,
   <gurobi.Var x1_1_11 (value -0.0)>,
   <gurobi.Var x1_1_12 (value -0.0)>,
   <gurobi.Var x1_1_13 (value -0.0)>,
   <gurobi.Var x1_1_14 (value -0.0)>,
   <gurobi.Var x1_1_15 (value -0.0)>,
   <gurobi.Var x1_1_16 (value -0.0)>,
   <gurobi.Var x1_1_17 (value -0.0)>,
   <gurobi.Var x1_1_18 (value -0.0)>,
   <gurobi.Var x1_1_19 (value -0.0)>,
   <gurobi.Var x1_1_20 (value -0.0)>,
   <gurobi.Var x1_1_21 (value -0.0)>,
   <gurobi.Var x1_1_22 (value -0.0)>,
   <gurobi.Var x1_1_23 (value -0.0)>,
   <gurobi.Var x1_1_24 (value -0.0)>,
   <gurobi.Var x1_1_25 (value -0.0)>,
   <gurobi.Var x1_1_26 (value 0.0)>],
  [<gurobi.Var x1_2_1

### objective value 結果

instance 1's optimal objective value is 1277269523.5

instance 2's optimal objective value is 163814977.29999998

instance 3's optimal objective value is 529130231.79999965

instance 4's optimal objective value is 

instance 5's optimal objective value is 